In [ ]:
#Load necessary packages
import pandas as pd
import os
import numpy as np
import copy

In [ ]:
#Load data created in Library_Week1_Notebook2_DataCleanUp

#Load data
os.chdir("//Users/Olga/Documents/INSIGHT2019/Library data/AllPldData")
Files_in_folder = os.listdir()
File_names = ['_2014', '_2015', '_2016', '_2001', '_2002', '_2003', 
              '_2004','_2005', '_2006', '_2007', '_2008', '_2009', 
              '_2010', '_2011', '_2012', '_2013', '_2000', '_1992', 
              '_1993', '_1994', '_1995', '_1996', '_1997', '_1998', '_1999']
Files = []

for filename in os.listdir():
    if filename.endswith('csv'):
        Files.append(pd.read_csv(filename, encoding = 'latin-1', low_memory = False))
        
#create dictionary of libraries labeled by year
LibData_dict = {}
for i in range(0, len(File_names)):
    LibData_dict[File_names[i]] = Files[i]
    
for k, v in LibData_dict.items():
    print(k, v.shape, type(LibData_dict[k]))

ColumnsPresent = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict.items():
    dataframe_to_join = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent = ColumnsPresent.join(dataframe_to_join, how = 'outer')

#Rearrange columns in dataframe. 
cols = ColumnsPresent.columns.tolist()
cols_rearr = cols[17:] + cols[0:17]
cols_rearr2 = cols_rearr[0:8] + cols_rearr[24:25] + cols_rearr[11:24] + cols_rearr[8:11]

ColumnsPresent_rearr = ColumnsPresent[cols_rearr2]

ColumnsPresent_rearr_str = ColumnsPresent_rearr.applymap(str)
ColumnsPresent_rearr_str = ColumnsPresent_rearr_str.reset_index(drop = True)

#create DF of unique column values
UniqueLabels = np.unique(ColumnsPresent_rearr_str.values)
LabelsBoolDF = pd.DataFrame(UniqueLabels)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_rearr_str.columns)):
    lis = []
    for i in range(len(UniqueLabels)):
        lis.append(ColumnsPresent_rearr_str.iloc[:, j].isin([UniqueLabels[i]]).any())
    LabelsBoolDF[ColumnsPresent_rearr_str.columns[j]] = lis
    
## Sum all TRUE values. Value of 25 = all true
sum_list = []
for i in range(LabelsBoolDF.shape[0]):
    sum_list.append(sum(LabelsBoolDF.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF['TRUE SUM'] = sum_list
LabelsBoolDF['Notes'] = ""
LabelsBoolDF.loc[LabelsBoolDF[LabelsBoolDF['TRUE SUM']==25].index, 'Notes'] = "OK"

#rows in which not all TRUE (meaning this column/data does not exist for all year)
MissingData = LabelsBoolDF[LabelsBoolDF['TRUE SUM']!=25].index.tolist()

#Columns in which data is unavailable for all years.
LabelsBoolDF.iloc[1,27] = "OK - Mailing address. Not necessary for analysis."
LabelsBoolDF.iloc[2,27] = "OK - Address type. Not necessary for analysis."
LabelsBoolDF.iloc[3,27] = "ATTEND was called VISITS after 1998. Combine with VISITS."
LabelsBoolDF.iloc[4:7,27] = "AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
LabelsBoolDF.iloc[12,27] = "Capital revenue, only computed after 2003. No equivalent prior, it seems."
LabelsBoolDF.iloc[13,27] = "OK - Call Based Statistical Area. Not necessary for analysis"
LabelsBoolDF.iloc[14,27] = "OK - Congressional district code. Not necessary for analysis."
LabelsBoolDF.iloc[15,27] = "OK - Census block. Not necessary for analysis."
LabelsBoolDF.iloc[17,27] = "OK - Census track. Not necessary for analysis."
LabelsBoolDF.iloc[19,27] = "OK - CITY_M. Not necessary for analysis."
LabelsBoolDF.iloc[21,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[22,27] = "County population. Incomplete. Probably OK. Just need to make sure the there is another better measure of population - POPU_LSA I think."
LabelsBoolDF.iloc[24,27] = "OK - FSCS Public Library Definition. Not necessary for analysis."
LabelsBoolDF.iloc[25,27] = "C_LEGBAS - Legal Basis Code. SAME as C_LEGBASE 1998 and ON. Combine the two rows."
LabelsBoolDF.iloc[26,27] = "C_LEGBASE - Legal Basis Code. SAME as C_LEGBAS 1997 and prior. Combine the two rows."
LabelsBoolDF.iloc[28,27] = "DATABASE - Total licensed databases."
LabelsBoolDF.iloc[29,27] = "DB_LOC - Local licensed databases"
LabelsBoolDF.iloc[30,27] = "DB_LO_OT - Local/other databases"
LabelsBoolDF.iloc[31:33,27] = "More database info"
LabelsBoolDF.iloc[33,27] = "DUPLI. Called HRS_OPEN 1998 ON. Combine with HRS_OPEN."
LabelsBoolDF.iloc[34,27] = "EBOOK. Starts 2003 (makes sense)"
LabelsBoolDF.iloc[35,27] = "EC_LO_OT. Local/other electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[36,27] = "EC_ST. State electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[37,27] = "ELACCEXP. Operating expendature for electronic access."
LabelsBoolDF.iloc[38,27] = "ELCONT. 2016 only [2016 doc: first collection]. Electronic content use. Total circulation."
LabelsBoolDF.iloc[39,27] = "ELECCOLL. Total electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[40,27] = "ELINFO. Retrival of electronic infomation. 2016 only. "
LabelsBoolDF.iloc[41,27] = "ELMATCIRC. Circulation of electronic material. 2013-2016."
LabelsBoolDF.iloc[42,27] = "ELMATEXP. 1995-2016. Circulation of electronic materials. This is a good one to use!"
LabelsBoolDF.iloc[43,27] = "ELMATS. Number of library materials in electronic format. 1995-2002"
LabelsBoolDF.iloc[44,27] = "ELSVCACC. Library access to electronic services. Yes, no. Unnecessary. 1995-2002."
LabelsBoolDF.iloc[45,27] = "OK - ENDDATE. Not necessary for analysis."
LabelsBoolDF.iloc[46,27] = "ERES_USR. 1999-2005. Electronic resource users per year. "
LabelsBoolDF.iloc[47,27] = "ESUBSCRP. 2003-2009. Electronic subscriptions."
LabelsBoolDF.iloc[48,27] = "FCAP_REV. Federal gov capital revenue. 2005 and on."
LabelsBoolDF.iloc[50:61,27] = "OK - FILL. Not necessary for analysis."
LabelsBoolDF.iloc[61,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[62,27] = "OK - place code. Not necessary for analysis."
LabelsBoolDF.iloc[63,27] = "OK - state code. Not necessary for analysis."
LabelsBoolDF.iloc[65:133,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[133,27] = "OK - GAL. Not necessary for analysis. Geocode location."
LabelsBoolDF.iloc[134,27] = "OK - GALMS. Not necessary for analysis."
LabelsBoolDF.iloc[135,27] = "GEOCODE - Geographical code. Might be useful. Can add to old datasets that don't have it."
LabelsBoolDF.iloc[136,27] = "OK - ID. Not necessary."
LabelsBoolDF.iloc[137,27] = "GPTERMS. Internet computers used by general public. 1998-2016."
LabelsBoolDF.iloc[138,27] = "HRS_OPEN. Called DUPLI 1997 and prior. Combine with DUPLI."
LabelsBoolDF.iloc[139:181,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[181,27] = "OK - County code. Not necassary for analysis."
LabelsBoolDF.iloc[182,27] = "OK - code. Not necassary for analysis."
LabelsBoolDF.iloc[183,27] = "INETACC - Internet access."
LabelsBoolDF.iloc[184,27] = "OK - imputation flag"
LabelsBoolDF.iloc[185,27] = "Kid program attendance. Combine with KIDATTEN."
LabelsBoolDF.iloc[186,27] = "Kid program attendance. Combine with KIDATTEND."
LabelsBoolDF.iloc[188,27] = "KIDPRO - Total kids program attendance."
LabelsBoolDF.iloc[189,27] = "OK - LATITUDE. NOt necessary for analysis."
LabelsBoolDF.iloc[190,27] = "LCAP_REV. Local gov capital revenue."
LabelsBoolDF.iloc[193,27] = "LIBRARIAN. Combine with LIBRARIA."
LabelsBoolDF.iloc[194,27] = "LIBRARIA. Combine with LIBRARIAN."
LabelsBoolDF.iloc[197,27] = "Urban-centric locale code. 2008-2016. 2008 doc: started in 2008. Can add to older years if necessary."
LabelsBoolDF.iloc[199,27] = "OK - LOGITUDE. Not necessary for analysis."
LabelsBoolDF.iloc[200,27] = "OK - LSABOUND. Legal services boundary. Not necessary for analysis."
LabelsBoolDF.iloc[202,27] = "OK - MAT_CENT. Match centroid. Not necessary for analysis."
LabelsBoolDF.iloc[203,27] = "OK - MAT_TYPE. Match status code. Not necessary for analysis."
LabelsBoolDF.iloc[204,27] = "OK - MICROF. Metro, micropolitan. Not necessary for analysis."
LabelsBoolDF.iloc[205,27] = "OK - MSTATUS. Match status. Not necessary for analysis." 
LabelsBoolDF.iloc[207,27] = "OCAP_REV. Other capital revenue."
LabelsBoolDF.iloc[209,27] = "OTHMATEX. Operating expenditures for all other library materials."
LabelsBoolDF.iloc[213,27] = "PHYSCIR. First collected in 2016."
LabelsBoolDF.iloc[214,27] = "PITUSR. Public use of internet."
LabelsBoolDF.iloc[215,27] = "POPU. Combine with POPU_LSA."
LabelsBoolDF.iloc[216,27] = "POPU_LSA. Population of legal service area. Combine with POPU."
LabelsBoolDF.iloc[217,27] = "POPU_UND. Combine with POPU_UNDUP."
LabelsBoolDF.iloc[218,27] = "POPU_UNDUP. Combine with POPU_UND."
LabelsBoolDF.iloc[219,27] = "OK - POSTMS. Postal match status. Unnecessary for analysis."
LabelsBoolDF.iloc[220,27] = "PRIMATEXP - Print material expenditures."
LabelsBoolDF.iloc[221,27] = "PUB_FIPS - OK. State code. Not necessary for analysis."
LabelsBoolDF.iloc[222,27] = "REAPLOCALE - urbancentric location code. Only 2016. Anything similar used before?"
LabelsBoolDF.iloc[223,27] = "REFERENC. Combine with REFERENCE."
LabelsBoolDF.iloc[224,27] = "REFERENCE. Combine with REFERENC."
LabelsBoolDF.iloc[225,27] = "REGBOR. Registered users. Anything similar prior to 2006?"
LabelsBoolDF.iloc[228,27] = "SCAP_REV. State revenue."
LabelsBoolDF.iloc[229,27] = "OK - SCORE. Address match. Not necessary for analysis."
LabelsBoolDF.iloc[231,27] = "STAFFEXP - Total staff exp. Salary and benefits. Combine with TOTEXP."
LabelsBoolDF.iloc[232,27] = "OK - STARTDAT. Reporting period start date. Not necessary for analysis."
LabelsBoolDF.iloc[233,27] = "OK - STATADDR. Address change code. Not necessary for analysis."
LabelsBoolDF.iloc[234,27] = "OK - STATNAME. Name change code. Not necessary for analysis."
LabelsBoolDF.iloc[235,27] = "OK - STATSTRU. Structure change code. Not necessary for analysis."
LabelsBoolDF.iloc[236,27] = "OK - STFTERMS. Internal terminals used by staff only. Not necessary for analysis."
LabelsBoolDF.iloc[238,27] = "SUBSCRIP. Combine with SUBSCRIPT."
LabelsBoolDF.iloc[239,27] = "SUBSCRIPT. Combine with SUBSCRIP."
LabelsBoolDF.iloc[240,27] = "TOTATTEN. Total attendance at library programs."
LabelsBoolDF.iloc[242,27] = "TOTCOLL. Total collection use. Only 2016."
LabelsBoolDF.iloc[243,27] = "TOTEXP. TOtal staff expenditures. Salary and benefits. Combine with STAFFEXP."
LabelsBoolDF.iloc[244,27] = "TOTEXPCO. Total expenditure of lib collection. Combine with TOTEXPCOL."
LabelsBoolDF.iloc[245,27] = "TOTEXPCOL. Total expenditure of lib collection. Combine with TOTEXPCO."
LabelsBoolDF.iloc[247,27] = "TOTOPEXP. Total operating expenditures. Combine with TOTOPEXP1."
LabelsBoolDF.iloc[248,27] = "TOTOPEXP1. Total operating expenditures. Combine with TOTOPEXP."
LabelsBoolDF.iloc[249,27] = "TOTPEMP - Total paid FTE employees. Combine with TOTSTAFF."
LabelsBoolDF.iloc[250,27] = "TOTPRO - Total library programs. Can this be combined with another label?"
LabelsBoolDF.iloc[251,27] = "TOTSTAFF. Total paid FTE employees. Combine with TOTPEMP."
LabelsBoolDF.iloc[252:255,27] = "VIDEO was later split into VIDEO_DL and VIDEO_PH."
LabelsBoolDF.iloc[255,27] = "VISITS. Combine with ATTEND."
LabelsBoolDF.iloc[256,27] = "OK - WEB_ADDR. Not necessary for data."
LabelsBoolDF.iloc[257,27] = "WIFISESS. Any similar metric prior?"
LabelsBoolDF.iloc[258,27] = "YAATTEN. Young adult program attendance."
LabelsBoolDF.iloc[259,27] = "YAPRO. Total young adult programs."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[262,27] = "ZIP. Combine with ZIP1."
LabelsBoolDF.iloc[263,27] = "ZIP1. Combine with ZIP."
LabelsBoolDF.iloc[264:268,27] = "OK - More zip information. Not necessary for analysis."
LabelsBoolDF.iloc[268,27] = "Note that lack of nan."

#Column names to keep
#LabelsBoolDF.loc[[0, 3:13, 15], 0].tolist()
Cols_to_keep_idx = [0]
Cols_to_keep_idx.extend(range(3,13))
Cols_to_keep_idx.extend((16, 18, 20, 25, 26))
Cols_to_keep_idx.extend(range(33, 49))
Cols_to_keep_idx.extend((64, 135, 137, 138, 183))
Cols_to_keep_idx.extend(range(185, 189))
Cols_to_keep_idx.extend(range(191, 199))
Cols_to_keep_idx.append(201)
Cols_to_keep_idx.extend(range(206, 219))
Cols_to_keep_idx.append(220)
Cols_to_keep_idx.extend(range(222, 229))
Cols_to_keep_idx.extend((230, 231, 237))
Cols_to_keep_idx.extend(range(237, 256))
Cols_to_keep_idx.extend(range(257, 260))
Cols_to_keep_idx.extend((262, 263))

Cols_to_keep_in_lib_dfs = LabelsBoolDF.iloc[Cols_to_keep_idx, 0].tolist()

#Make dictionaries of smaller dataframe with Cols_to_keep_in_lib_dfs ONLY
LibData_dict_reduced = dict()

for k, v in LibData_dict.items():
    LibData_dict_reduced[k] = pd.DataFrame()
    for i in Cols_to_keep_in_lib_dfs:
        if i in v:
            TempList = list(v[i])
            LibData_dict_reduced[k][i] = TempList

#Did above loop do what I thought it did?
print("Original dfs:")
for k, v in LibData_dict.items():
    print(k, v.shape)

print("\n", "Reduced dfs:")  
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Combine col names that mean same thing in different years into single 
Names_to_combine = ["ATTEND", "VISITS",  "C_LEGBAS", "C_LEGBASE", "DUPLI", "HRS_OPEN", "KIDATTEN", "KIDATTEND", 
                    "LIBRARIAN", "LIBRARIA", "POPU", "POPU_LSA", "POPU_UND", "POPU_UNDUP", "REFERENC", "REFERENCE", 
                    "STAFFEXP", "TOTEXP", "SUBSCRIP", "SUBSCRIPT", "TOTEXPCO", "TOTEXPCOL", 
                    "TOTOPEXP", "TOTOPEXP1", "TOTPEMP", "TOTSTAFF", "ZIP", "ZIP1"]

for i in range(0, len(Names_to_combine)-1, 2):
    for k, v in LibData_dict_reduced.items(): 
        if Names_to_combine[i] in v:
            name = Names_to_combine[i]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name: newname}, inplace = True)
        elif Names_to_combine[i+1] in v:
            name = Names_to_combine[i + 1]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name : newname}, inplace = True)

#Check that above code worked - names changes
for k, v in LibData_dict_reduced.items():
    print(v.columns)

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Columns to add to DFs
"AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
"VIDEO was later split into VIDEO_DL and VIDEO_PH."

for k, v in LibData_dict_reduced.items(): 
    if "AUDIO_DL" in v:
        v["AUDIO"] = v['AUDIO_DL'] + v['AUDIO_PH']
    if "VIDEO_DL" in v:
        v["VIDEO"] = v['VIDEO_DL'] + v['VIDEO_PH']

